In [0]:
import requests
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
# Objective website
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
URL = 'https://www.cervezasonline.com/'
session = requests.Session()

In [80]:
#First we want to check the robots.txt file of our objective website
#A function get_robot_txt is constructed to check any url
def get_robot_txt(url):
    if url.endswith('/'):
        path = url
    else:
        path = url + '/'
    req = requests.get(path + "robots.txt",data =None)
    return req.text
# Read robots.txt file
# No product pages within any Disallow
print(get_robot_txt(URL))

# robots.txt automaticaly generated by PrestaShop e-commerce open-source solution
# http://www.prestashop.com - http://www.prestashop.com/forums
# This file is to prevent the crawling and indexing of certain parts
# of your site by web crawlers and spiders run by sites like Yahoo!
# and Google. By telling these "robots" where not to go on your site,
# you save bandwidth and server resources.
# For more information about the robots.txt standard, see:
# http://www.robotstxt.org/robotstxt.html
User-agent: *
# Allow Directives
Allow: */modules/*.css
Allow: */modules/*.js
# Private pages
Disallow: /*?orderby=
Disallow: /*?orderway=
Disallow: /*?tag=
Disallow: /*?id_currency=
Disallow: /*?search_query=
Disallow: /*?back=
Disallow: /*?n=
Disallow: /*&orderby=
Disallow: /*&orderway=
Disallow: /*&tag=
Disallow: /*&id_currency=
Disallow: /*&search_query=
Disallow: /*&back=
Disallow: /*&n=
Disallow: /*controller=addresses
Disallow: /*controller=address
Disallow: /*controller=authentication
Disall

In [0]:
# Class Category. All of the identified categories have one of these
class Category:
  def __init__(self, category_url=None):
    if category_url[-1] == '#': self.url = category_url[:-1]
    else: self.url = category_url    

  # Identify if there is a 'Show All' option
  def get_all_products_web_page(self):
    soup = BeautifulSoup(session.get(self.url, headers=headers).content, 'html.parser')
    try: 
      # Identify number of All products in "Mostrar Todos" button
      value = soup.find(id='pagination').find(id='nb_item')['value']
      # Create new URL with the new parametres of number of producst as n
      self.url = self.url+'?n='+value
    except: pass

  # Identify all the products in a category page
  def get_all_product_links(self):
    # Get page with all products from category
    self.get_all_products_web_page()
    soup = BeautifulSoup(session.get(self.url, headers=headers).content, 'html.parser')
    # Get all product links one by one from html code
    product_links = [product['href'] for product in soup.findAll('a', class_='product_img_link')]
    print("Category URL: ", self.url)
    print("Number of products in category: ", len(product_links))
    return(product_links)

In [0]:
# This method returns all producst identified in the website
def get_all_products(url):
  # Identify all categories on the top block div
  soup = BeautifulSoup(session.get(url, headers=headers).content, 'html.parser')
  # Get all identified categories
  # Detele all categories that are longer than one subdomain
  categories = ['/'.join((url + category['href'][1:]).split('/')[0:4]) for category in soup.find(id='block_top_menu').find_all('a') if category['href']!='#' and category['href'][0]=='/']
  # Create categories with only unqiue urls
  categories = [Category(category) for category in list(set(categories))]
  # Get all the links to products per cateogry
  products = []
  [products.extend(category.get_all_product_links()) for category in categories]
  # Get only unique URLs
  products = list(set(products))
  print("Total number of products: ", len(products))
  return(products)

In [0]:
## TODO:
  # Call function get_all_products() to get all products from 'https://www.cervezasonline.com/' website
  # Parse all of the products in the website

In [0]:
df = pd.DataFrame()
URL = ['https://www.cervezasonline.com/309-botellones','https://www.cervezasonline.com/304-latas',
       'https://www.cervezasonline.com/341-botellon-metal','https://www.cervezasonline.com/354-barriles',
      'https://www.cervezasonline.com/302-botella-cerveza#/page-*']

def substring_after(s, delim):
    return s.partition(delim)[2]


for url in URL:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    #print titles only
    results = soup.find(id='category')
    job_elems = results.find_all('div', class_='right-block')

    l = []

    for job_elem in job_elems:
        product_name = job_elem.find('a', class_='product-name')
        product_desc = job_elem.find('p', class_='product-desc')
        product_price = job_elem.find('span', class_='price product-price')
        l.append([substring_after(url,"https://www.cervezasonline.com/")[4:].upper(),
              product_name.text.strip().replace(product_name.text.strip()[product_name.text.strip().rindex(' ')+1:],"").strip(),
              product_name.text.strip()[product_name.text.strip().rindex(' ')+1:].upper().replace(",","."),
              substring_after(product_desc.text.strip(),"Origen:").
              replace(substring_after(product_desc.text.strip(),"Tipo:"),"").replace("Tipo:","").strip(),
              
              substring_after(product_desc.text.strip(),"Tipo:").
              replace(substring_after(product_desc.text.strip(),"Color:"),"").replace("Color:","").strip(),
              
              substring_after(product_desc.text.strip(),"Color:").
              replace(substring_after(product_desc.text.strip(),"Graduación:"),"").replace("Graduación:","").strip(),
              
              substring_after(product_desc.text.strip(),"Graduación:"),
              product_price.text.replace(",",".").replace(" ","")])

    df = df.append(pd.DataFrame(l,columns = ['Categoria','Nombre de la Cerveza', 'Tamaño','Origen','Tipo','Color',
                                             'Graduación','Precio'])).reset_index(drop=True)


    
df.loc[df['Tamaño'].str.contains('CL'), 'Tamaño en litros'] = df['Tamaño'].replace("CL:","")


In [0]:
df.loc[df['Tamaño'].str.contains('CL'), 'Tamaño en litros'] = pd.to_numeric(df['Tamaño'].
                                                                          map(lambda x: x.strip('+-').rstrip('CL')))/100

df['Tamaño en litros'] = np.where(df['Tamaño en litros'].isna(), 
                                  df['Tamaño'].map(lambda x: x.strip('+-').rstrip('L')),df['Tamaño en litros'])

df['Tamaño en litros'] = pd.to_numeric(df['Tamaño en litros'])
df['Precio por litro'] = pd.to_numeric(df['Precio'].map(lambda x: x.strip('+-').rstrip('€')))/df['Tamaño en litros']

df.head(10)

,Categoria,Nombre de la Cerveza,Tamaño,Origen,Tipo,Color,Graduación,Precio,Tamaño en litros,Precio por litro
0,BOTELLONES,La Chouffe,75CL,Bélgica,Ale,Rubia dorada,8º,5.05€,0.75,6.733333
1,BOTELLONES,Chimay Azul,75CL,Bélgica,Trapense,Marrón oscuro,9º,5.80€,0.75,7.733333
2,BOTELLONES,Augustijn,3L,Bélgica,Abadía,Rubio intenso,7º,48.10€,3.00,16.033333
3,BOTELLONES,Augustijn,1.5L,Bélgica,Abadía,Rubio intenso,7º,19.10€,1.50,12.733333
4,BOTELLONES,Triple Karmeliet,1.5L,Bélgica,Especial,Dorado,8.4º,21.75€,1.50,14.500000
5,BOTELLONES,Big Chouffe,1.5L,Bélgica,Ale,Rubia dorada,8º,16.70€,1.50,11.133333
6,BOTELLONES,Chimay Azul,1.5L,Bélgica,Trapense,Marrón oscuro,9º,24.85€,1.50,16.566667
7,BOTELLONES,Delirium Nocturnum,75CL,Bélgica,Ale,Oscura,8.5º,5.70€,0.75,7.600000
8,BOTELLONES,Duvel,1.5L,Bélgica,Ale,Rubia dorada,8.5º,26.45€,1.50,17.633333
9,BOTELLONES,Mac Chouffe,75CL,Bélgica,Ale,Oscuro,8.5º,5.20€,0.75,6.933333


In [0]:
df.describe()

,Tamaño en litros,Precio por litro
count,92.000000,92.000000
mean,1.317609,7.942922
std,4.328042,4.340511
min,0.250000,1.350000
25%,0.330000,5.187879
50%,0.500000,6.666667
75%,0.750000,9.919697
max,30.000000,22.000000


In [0]:
import plotly.graph_objects as go
from collections import Counter

fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="count", y=df['Nombre de la Cerveza'], x=df['Origen'], name="COUNT")).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title="Número de cervezas por origen"
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="count", y=df['Nombre de la Cerveza'], x=df['Tipo'], name="COUNT")).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title="Número de cervezas por tipo"
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="count", y=df['Precio por litro'], x=df['Precio por litro'], name="COUNT")).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title="Precio de la cerveza por litro (en €)"
)
fig.show()

In [0]:
df.to_csv(r'C:\Users\guillermo.arguello\Documents\Master Data Science UOC\Tipología y ciclo de vida de los datos\Pec1 2020\pec_1_web_scrapping.csv')
